In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

def load(path: str) -> pd.DataFrame:
    """Load a CSV file into a Dataset object.

    Args:
        path (str): path to the CSV file

    Returns:
        Dataset: object containing the data
    """
    try:
        if not path.lower().endswith(("csv")):
            raise AssertionError("Only csv formats are supported.")
        local_dir = os.path.abspath('')
        file_path = os.path.join(local_dir, path)
        if not os.path.exists(file_path) or os.path.isdir(file_path):
            raise AssertionError("File not found:", file_path)
        df = pd.read_csv(file_path)
        return df
    except AssertionError as error:
        print(f"{AssertionError.__name__}: {error}")
        return None

In [2]:
train_knight = load("/home/lcompieg/sgoinfre/Train_knight.csv")
test_knight = load("/home/lcompieg/sgoinfre/Test_knight.csv")

In [3]:
def voting(train_knight: pd.DataFrame, test_knight: pd.DataFrame) -> None:
    x = train_knight[['Power', 'Reactivity', 'Push', 'Lightsaber', 'Repulse', 'Blocking', 'Deflection', 'Evade', 'Combo', 'Attunement']]
    # x = train_knight[['Hability', 'Agility', 'Reactivity', 'Push', 'Survival', 'Blocking', 'Deflection', 'Mass', 'Sprint', 'Attunement']]
    y = train_knight['knight']
    
    scaled = StandardScaler()
    x = scaled.fit_transform(x)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    
    base_learners = [
        ('logistic', LogisticRegression(solver='lbfgs')),
        ('knn', KNeighborsClassifier(n_neighbors=2)),
        ('decision_tree', DecisionTreeClassifier(random_state=42))
    ]
    
    vtg = VotingClassifier(estimators=base_learners, voting='hard')
    
    vtg.fit(x, y)
    y_pred = vtg.predict(X_test)
    
    print(f"F1_score : {f1_score(y_test, y_pred, average='macro')}")
    
    x_test =  test_knight[['Power', 'Reactivity', 'Push', 'Lightsaber', 'Repulse', 'Blocking', 'Deflection', 'Evade', 'Combo', 'Attunement']] 
    # x_test = train_knight[['Hability', 'Agility', 'Reactivity', 'Push', 'Survival', 'Blocking', 'Deflection', 'Mass', 'Sprint', 'Attunement']]
    x_test = StandardScaler().fit_transform(x_test)
    
    y_pred_test = vtg.predict(x_test)
    
    with open("Voting.txt", "w") as output:
          for item in y_pred_test:
            output.write(item + "\n")
  

In [4]:
voting(train_knight, test_knight)

F1_score : 1.0
